## 05. Сравниваем модели

В этом ноутбуке мы попробуем сравнить результаты трех моделей:

* Необученного Yandex GPT
* Yandex GPT с RAG
* Fine-tuned Yandex GPT

В качестве самостоятельного (достаточно простого) упражнения попробуйте также добавить вариант Fine-Tuned Yandex GPT + RAG.

Для поддержки fine-tuned модели нам потребуется библиотека [yandex-chain](http://github.com/yandex-datasphere/yandex-chain):

In [5]:
import os

from yandex_chain import YandexLLM

gpt = YandexLLM(folder_id="", api_key=os.environ["api_key"])
gpt("Как произносится заклинание невидимости?")

'Произносить заклинание невидимости будет легче и понятнее, если вы возьмете на себя труд выучить рифмы каждого заклинания. Для заклинания невидимости они\xa0будут следующими:\n\nI summon shadows, gather in the night\nCome over land, deep within my sight\nYou may behold me, even if unseen\nSwap form I will, yet remain the same. (3)\nМногие\xa0заклинания прописываются на\xa0английском языке, так как он является наиболее распространённым в\xa0мире магии, но также можно переводить их\xa0на русский или любой другой язык. Существует большая часть заклинаний для красоты.'

Теперь попробуем тоже самое для fine-tuned модели. Для этого нам понадобится знать специальный ключ **instruction uri**, который можно передать в объект `YandexLLM`.

> Чтобы не возникало проблем с доступом, сервисный аккаунт, от имени которого вы ходите в Yandex GPT, должен быть добавлен в проект DataSphere, в котором вы осуществляли доучивание модели!

In [16]:
instruction_uri = "ds://bt1s5o0jbv9avtidu9ka"
fgpt = YandexLLM(
    folder_id="", instruction_uri=instruction_uri, api_key=os.environ["api_key"]
)
fgpt.temperature = 0.4
fgpt("Как произносится заклинание невидимости?")

'заклинание невидимости произносится как «ассио».'

Теперь для сравнения быстренько соберём конвейер Retrieval-Augmented Generation из ноутбука 03:

In [20]:
import lancedb
import langchain.chains
import langchain.prompts
from langchain.vectorstores import LanceDB
from yandex_chain import YandexEmbeddings

db_dir = "store"
db = lancedb.connect(db_dir)
table = db.open_table("vector_index")
embeddings = YandexEmbeddings(folder_id="", api_key=os.environ["api_key"])
vec_store = LanceDB(table, embeddings)
retriever = vec_store.as_retriever(search_kwargs={"k": 5})

# Промпт для обработки документов
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
Текст:
-----
{context}
-----
Вопрос:
{query}"""
prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=gpt, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)


def answer(query):
    results = retriever.get_relevant_documents(query)
    return chain.run(input_documents=results, query=query)


answer("Как произносится заклинание невидимости?")

'Это непростой вопрос. Предлагаю следующие варианты произношения: антАйгчесинг спилл\nантАигачнас пилл;\xa0спилл\nанти исДжесинг точка пилв\nантэйческий спилл.'

Теперь сделаем функцию, которая выводит для сравнения сразу три ответа на поставленный вопрос. Мы также дадим возможность задавать температуру моделей для генерации:

In [21]:
def compare(q, temperature=0.1):
    gpt.temperature = temperature
    fgpt.temperature = temperature
    print(f"Ответ YGPT: {gpt(q)}")
    print(f"Ответ RAG: {answer(q)}")
    print(f"Ответ Fine-Tune: {fgpt(q)}")


compare("Что означает Вингардиум Левиоса")

Ответ YGPT: Вингардиум Левиоса (англ. Wingardium Leviosa) — заклинание, вызывающее левитацию предмета.

В книге «Гарри Поттер и философский камень» Гарри Поттер, пытаясь научиться заклинанию левитации, произносит «Вингардиум Левиоса», но ничего не происходит. Профессор МакГонагалл объясняет ему, что для того, чтобы заклинание сработало, необходимо верить в то, что оно сработает. После этого у Гарри получается поднять перо в воздух.

В фильме «Гарри Поттер и философский камень» роль заклинания «Вингардиум Левиоса» играет заклинание «Локомотор», вызывающее движение предмета в заданном направлении.
Ответ RAG: Вингардиум Левиоса — это заклинание, которое позволяет поднимать предметы в воздух.
Ответ Fine-Tune: Заклинание Левитации


Оставляем на ваше самостоятельное усмотрение дальнейшие эксперименты с этими моделями! Поговорите с ними!